# Linear Reissner-Mindlin shell

Reference values for free boundary condition: 

$t=0.1$: $0.18954566$, 

$t=0.01$: $0.15046617$, 

$t=0.001$: $0.1498902$

In [ ]:
from ngsolve import *
import netgen.meshing as meshing
from ngsolve.webgui import Draw

In [ ]:
thickness = 0.1
order = 3

# Material properties
E   = 2.85e4
nu  = 0.3
mu  = E/(2*(1+nu))
lam = E*nu/(1-nu**2)
G   = E/(2*(1+nu))
k   = 5/6


free = "right"
# only 1/8 of the shell is considered due to symmetry
symmetry ="left|top|bottom"

# periodic force
zeta = IfPos(y, atan(z/y), pi/2)
e_n = specialcf.normal(3)
force = 1e4*thickness*cos(2*zeta)*e_n*thickness**2

# geometry
mapping = lambda x,y,z : (x,sqrt(1+x**2)*cos(pi/2*y),sqrt(1+x**2)*sin(pi/2*y))
geom = meshing.SurfaceGeometry(mapping)
mesh = Mesh(geom.GenerateMesh(quads=False, nx=5, ny=5)).Curve(order)
Draw(mesh)

In [ ]:
interpolateShear=True
def MaterialStress(mat):
    return E/(1-nu**2)*((1-nu)*mat+nu*Trace(mat)*Id(3))

t = thickness
       
fesU = VectorH1(mesh, order=order, dirichletx_bbnd="left", dirichlety_bbnd="top", dirichletz_bbnd="bottom")
fesB = VectorH1(mesh, order=order,orderface=order+1 if order < 4 else order, \
                dirichletx_bbnd="left", dirichlety_bbnd="top", dirichletz_bbnd="bottom")
fes = fesU*fesB 
(u,beta), (du,dbeta) = fes.TnT()

fesR = HCurl(mesh, order=order, type1=True)

n = specialcf.normal(3)
Ptau = Id(3) - OuterProduct(n,n)
gradu = Grad(u).Trace()
graddu = Grad(du).Trace()


a = BilinearForm(fes, symmetric=True)
a += (t*InnerProduct(MaterialStress(Sym(Ptau*gradu)), Sym(Ptau*graddu)))*ds    
a += (t**3/12*InnerProduct(MaterialStress(Sym(Ptau*Grad(beta).Trace() + gradu.trans*Grad(n))), \
                        Sym(Ptau*Grad(dbeta).Trace() + graddu.trans*Grad(n))))*ds
if interpolateShear:
    if order > 1:
        a += (t*k*G*InnerProduct( Interpolate(gradu.trans*n+beta,fesR), Interpolate(graddu.trans*n+dbeta,fesR) ))*ds
        a += (t*k*G*(n*beta)*(n*dbeta))*ds
    else:
        a += (t**3/(t**2+specialcf.mesh_size**2)*k*G*InnerProduct( Interpolate(gradu.trans*n+beta,fesR), Interpolate(graddu.trans*n+dbeta,fesR)))*ds
        a += (t**3/(t**2+specialcf.mesh_size**2)*k*G*(n*beta)*(n*dbeta))*ds
else:
    a += (t*k*G*InnerProduct(gradu.trans*n+beta, graddu.trans*n+dbeta))*ds
    
f = LinearForm(fes)
f += force*du*ds

In [ ]:
gfsol = GridFunction(fes)

with TaskManager():
    a.Assemble()
    f.Assemble()
    
    gfsol.vec.data = a.mat.Inverse(fes.FreeDofs(),inverse="sparsecholesky")*f.vec

In [ ]:
gfu, gfbeta = gfsol.components
print("value = ", gfu(mesh(0,0,1,BND))[2])
Draw(gfu, mesh, deformation=True)